In [146]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [147]:
!pip install imbalanced-learn --quiet


In [148]:
cd /content/drive/MyDrive/Works Sept-Dec 2023/Diabetes Prediction/code

/content/drive/MyDrive/Works Sept-Dec 2023/Diabetes Prediction/code


In [149]:
import pandas as pd

file_path = 'diabetes_data_upload.csv'
diabetes_data = pd.read_csv(file_path)

# Display the first few rows of the dataset
diabetes_data.head()


Age Gender Polyuria Polydipsia sudden weight loss weakness Polyphagia  \
0  40.0   Male       No        Yes                 No      Yes         No   
1  58.0   Male       No         No                 No      Yes         No   
2  41.0   Male      Yes         No                 No      Yes        Yes   
3  45.0   Male       No         No                Yes      Yes        Yes   
4  60.0   Male      Yes        Yes                Yes      Yes        Yes   

  Genital thrush visual blurring Itching Irritability delayed healing  \
0             No              No     Yes           No             Yes   
1             No             Yes      No           No              No   
2             No              No     Yes           No             Yes   
3            Yes              No     Yes           No             Yes   
4             No             Yes     Yes          Yes             Yes   

  partial paresis muscle stiffness Alopecia Obesity     class  
0              No              Yes      Yes     Yes  Positive  
1             Yes               No      Yes      No  Positive  
2              No              Yes      Yes      No  Positive  
3              No               No       No      No  Positive  
4             Yes              Yes      Yes     Yes  Positive

In [150]:
diabetes_data.dropna(inplace=True)

In [151]:
diabetes_data.columns

Index(['Age', 'Gender', 'Polyuria', 'Polydipsia', 'sudden weight loss',
       'weakness', 'Polyphagia', 'Genital thrush', 'visual blurring',
       'Itching', 'Irritability', 'delayed healing', 'partial paresis',
       'muscle stiffness', 'Alopecia', 'Obesity', 'class'],
      dtype='object')

In [152]:
diabetes_data['age_group'] = diabetes_data['Age'].apply(lambda age:  'below' if age<40 else 'above')
categorical_columns = diabetes_data.select_dtypes(include=['object']).columns.tolist()
categorical_columns

['Gender',
 'Polyuria',
 'Polydipsia',
 'sudden weight loss',
 'weakness',
 'Polyphagia',
 'Genital thrush',
 'visual blurring',
 'Itching',
 'Irritability',
 'delayed healing',
 'partial paresis',
 'muscle stiffness',
 'Alopecia',
 'Obesity',
 'class',
 'age_group']

In [153]:
categorical_columns.remove('Gender')
categorical_columns.remove('class')
categorical_columns.remove('age_group')
categorical_columns

['Polyuria',
 'Polydipsia',
 'sudden weight loss',
 'weakness',
 'Polyphagia',
 'Genital thrush',
 'visual blurring',
 'Itching',
 'Irritability',
 'delayed healing',
 'partial paresis',
 'muscle stiffness',
 'Alopecia',
 'Obesity']

In [154]:
for categorical_column in categorical_columns:
  diabetes_data[categorical_column] = diabetes_data[categorical_column].map({'Yes': 1, 'No': 0})

In [155]:
diabetes_data = pd.get_dummies(diabetes_data, columns=['Gender'], prefix=['Gender'])
diabetes_data['class'] = diabetes_data['class'].map({'Positive': 1, 'Negative': 0})
diabetes_data.columns

Index(['Age', 'Polyuria', 'Polydipsia', 'sudden weight loss', 'weakness',
       'Polyphagia', 'Genital thrush', 'visual blurring', 'Itching',
       'Irritability', 'delayed healing', 'partial paresis',
       'muscle stiffness', 'Alopecia', 'Obesity', 'class', 'age_group',
       'Gender_Female', 'Gender_Male'],
      dtype='object')

In [156]:
abovedf = diabetes_data[diabetes_data['age_group']=='above']
abovedf.drop('age_group', axis=1,inplace=True)
belowdf = diabetes_data[diabetes_data['age_group']=='below']
belowdf.drop('age_group', axis=1,inplace=True)

<ipython-input-156-c24eae1c1940>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abovedf.drop('age_group', axis=1,inplace=True)
<ipython-input-156-c24eae1c1940>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  belowdf.drop('age_group', axis=1,inplace=True)


In [157]:
abovedf.shape, belowdf.shape

((371, 18), (144, 18))

In [158]:
abovedf['class'].value_counts(), belowdf['class'].value_counts()

(1    231
 0    140
 Name: class, dtype: int64,
 1    85
 0    59
 Name: class, dtype: int64)

In [159]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.base import ClassifierMixin
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, KFold

def train_model(model, data):

  y = data['class']
  X = data.drop('class', axis=1)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)

  # Calculate classification metrics
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)

  # Create a DataFrame to display the metrics
  metrics_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Score': [accuracy, precision, recall, f1]
  })

  feature_importances = model.feature_importances_

  # Create a list of feature names from your dataset
  feature_names = list(X.columns)  # Replace X with your feature matrix

  # Create a dictionary with feature names and their importances
  feature_importance_dict = dict(zip(feature_names, feature_importances))

  # Sort the dictionary by importance in descending order
  sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

  # Print or return the sorted feature importances
  print(sorted_feature_importance)



In [160]:
abovedf.shape

(371, 18)

In [161]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
train_model(model, abovedf)

[('Polydipsia', 0.19478743326768858), ('Polyuria', 0.19411182305753816), ('Alopecia', 0.07977110501004354), ('Age', 0.07551869117712717), ('Gender_Male', 0.0664405985888745), ('Gender_Female', 0.05704318122185059), ('Irritability', 0.04572729576175815), ('partial paresis', 0.044672578608145344), ('delayed healing', 0.04306050518364337), ('sudden weight loss', 0.03959389163723594), ('Itching', 0.035735779589059026), ('Obesity', 0.024122988277376144), ('visual blurring', 0.02368219320429802), ('muscle stiffness', 0.020506969412221295), ('Genital thrush', 0.01884255123800371), ('Polyphagia', 0.018778312826989545), ('weakness', 0.017604101938146953)]


In [162]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
train_model(model, belowdf.copy())

[('Polyuria', 0.23506276162577694), ('Polydipsia', 0.147662408973663), ('Age', 0.08582241125977941), ('delayed healing', 0.08200076730897105), ('partial paresis', 0.07324904720666271), ('Alopecia', 0.05536469702142925), ('Gender_Male', 0.05051510196235229), ('Polyphagia', 0.039084973566804), ('visual blurring', 0.03814646659136716), ('muscle stiffness', 0.03606727661688411), ('Itching', 0.03551503814992096), ('Genital thrush', 0.03235410957366605), ('Gender_Female', 0.025950195474480693), ('weakness', 0.023523241239305045), ('sudden weight loss', 0.020076612786665837), ('Irritability', 0.010121506031662446), ('Obesity', 0.009483384610609125)]
